In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Symbols

---


| Variable                                 	| Unit of Measurement 	|
|------------------------------------------	|----------------------	|
| $k$, permeability of the liver           	| $m^2$                	|
| $\mu_s$, viscosity of embolizing solution 	| $Pa \cdot s$         	|
| $\mu_o$, viscosity of blood               	| $Pa \cdot s$         	|
| distances                                	| $m$                  	|

| **Symbol**                  | **Description**                                | **Unit**           |
|-----------------------------|------------------------------------------------|--------------------|
| $ k $                     | Permeability of the liver                      | $ \mathrm{m}^2 $ |
| $ \mu_s $                 | Viscosity of embolizing solution               | $ \mathrm{Pa} \cdot \mathrm{s} $ |
| $ \mu_o $                 | Viscosity of blood                             | $ \mathrm{Pa} \cdot \mathrm{s} $ |
| $ \Omega $                | 3D Bulk Domain                                 | Dimensionless      |
| $ \Sigma $                | Cylindrical Vessel                             | Dimensionless      |
| $ \Gamma $                | Lateral Boundary of $ \Sigma $               | Dimensionless      |
| $ \Gamma_0, \Gamma_S $    | Top and Bottom Faces of $ \Sigma $           | Dimensionless      |
| $ \lambda(s) $            | $\mathcal{C}^2$-regular Curve (Centerline)    | $ \mathrm{m} $ (Parameter $ s $) |
| $ s $                     | Arc-length Parameter along $ \lambda $        | $ \mathrm{m} $   |
| $ \Lambda $               | Centerline of $ \Sigma $                      | Dimensionless      |
| $ \mathcal{D}(s) $        | Cross-section of $ \Sigma $                   | $ \mathrm{m}^2 $ |
| $ \partial \mathcal{D}(s) $| Boundary of Cross-section $ \mathcal{D}(s) $  | $ \mathrm{m} $   |
| $ u_{\oplus} $            | Fluid Potential in $ \Omega $ (Exterior)       | $ \mathrm{Pa} $  |
| $ u_{\ominus} $           | Fluid Potential in $ \Lambda $ (Interior)      | $ \mathrm{Pa} $  |
| $ \beta $                 | Coupling Coefficient                            | $ \frac{\mathrm{m}^3}{\mathrm{s} \cdot \mathrm{Pa}} $ |
| $ \kappa $                | Permeability Coefficient                        | $ \frac{\mathrm{m}^2}{\mathrm{s} \cdot \mathrm{Pa}} $ |
| $ g $                     | Source Term                                    | $ \frac{\mathrm{m}}{\mathrm{s}} $ |
| $ \alpha $                | Diffusion Coefficient                           | $ \frac{\mathrm{m}^2}{\mathrm{s} \cdot \mathrm{Pa}} $ |
| $ \Delta $                | Laplacian Operator                              | $ \frac{1}{\mathrm{m}^2} $ |
| $ \delta_{\Lambda} $      | Dirac Measure on $ \Lambda $                  | $ \frac{1}{\mathrm{m}^2} $ |
| $ f $                     | Forcing Term                                    | $ \frac{1}{\mathrm{s}} $ |
| $ \bar{u}_{\oplus} $      | Averaged Fluid Potential in $ \Omega $         | $ \mathrm{Pa} $  |
| $ \bar{u}_{\ominus} $     | Averaged Fluid Potential in $ \Lambda $        | $ \mathrm{Pa} $  |

In [2]:
# @title Install nonstandard libraries
%%capture
!pip install ipywidgets
!pip install vtk
!pip install meshio
!pip install pyvista
!pip install Rtree

import os, re

def replace_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Replace 'ufl' with 'ufl_legacy'
    content = re.sub(r'\bufl\b', 'ufl_legacy', content)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                replace_in_file(file_path)

# dolfin
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"

# block
try:
    import block
except ImportError:
    !git clone "https://bitbucket.org/fenics-apps/cbc.block/src/master/"
    !pip install master/

# fenics_ii
try:
    import xii
except ImportError:
    !git clone "https://github.com/MiroK/fenics_ii"
    process_directory("fenics_ii/")
    !pip install fenics_ii/

# graphnics
try:
    import graphnics
except ImportError:
    !git clone "https://github.com/IngeborgGjerde/graphnics"
    !pip install graphnics/

In [6]:
WD_PATH = "/content/drive/MyDrive/Research/3D-1D"

import sys, os
sys.path.append(os.path.join(WD_PATH, 'modules'))

import FEMEdge
import FEMSink

import meshio
import scipy
import copy
import vtk
import json
import numpy as np
import matplotlib.pyplot as plt
import datetime
import networkx as nx
from matplotlib.gridspec import GridSpec
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import cKDTree
import importlib
from dolfin import *
from vtk.util.numpy_support import vtk_to_numpy
from xii import *
from graphnics import *

In [ ]:
# @title Define G = .vtk domain read
import vtk

def read_vtk(file_path):
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    output = reader.GetOutput()

    G = FenicsGraph()

    damage_array = output.GetPointData().GetArray("Damage")
    for i in range(output.GetNumberOfPoints()):
        point = output.GetPoint(i)
        damage_value = damage_array.GetValue(i)
        G.add_node(i, pos=tuple(point), damage=damage_value)

    radius_array = output.GetCellData().GetArray("Radius")
    for i in range(output.GetNumberOfCells()):
        cell = output.GetCell(i)
        point_ids = [cell.GetPointId(j) for j in range(cell.GetNumberOfPoints())]
        for j in range(len(point_ids) - 1):
            u = point_ids[j]
            v = point_ids[j + 1]
            radius_value = radius_array.GetValue(i) if radius_array else None
            G.add_edge(u, v, radius=radius_value)

    return G

# Usage
file_path = WD_PATH + '/data/vtk/sortedDomain.vtk'
# file_path = WD_PATH + '/oncopigReferenceData/ZPAF23S018/20230531/vesselNetwork_upDated.vtk'
G = read_vtk(file_path)

def filter_small_components(G, threshold=20):
  """
  Removes connected components from the graph G that have a total length
  (sum of edge lengths) less than the specified threshold.

  Args:
    G: The networkx graph to process.
    threshold: The minimum total length for a component to be kept.
  """

  # Convert to undirected graph
  G_undirected = G.to_undirected()

  # Find connected components
  components = list(nx.connected_components(G_undirected))

  # Calculate total length for each component
  for component in components:
    total_length = 0
    for u, v in G_undirected.subgraph(component).edges():
      pos_u = G_undirected.nodes[u]['pos']
      pos_v = G_undirected.nodes[v]['pos']
      length = ((pos_u[0] - pos_v[0])**2 +
                (pos_u[1] - pos_v[1])**2 +
                (pos_u[2] - pos_v[2])**2) ** 0.5
      total_length += length

    # Remove component if total length is below threshold
    if total_length < threshold:
      G.remove_nodes_from(component)  # Remove from the original directed graph

# Apply the filtering
filter_small_components(G)

# @title Clean graph (only for raw .vtk domain reads)
def cleanup_graph(G):
    G.remove_nodes_from(list(nx.isolates(G)))
    G = nx.convert_node_labels_to_integers(G)

    # Merge close nodes
    positions = nx.get_node_attributes(G, 'pos')
    positions_array = np.array(list(positions.values()))
    tree = cKDTree(positions_array)
    merged_nodes = set()

    for node, pos in positions.items():
        # Find nearby nodes within the threshold distance
        nearby_nodes = tree.query_ball_point(pos, 1.0e-4)

        # Merge with the first nearby node (if any) that hasn't been merged yet
        for other_node in nearby_nodes:
            if other_node != node and other_node not in merged_nodes:
                G = nx.contracted_nodes(G, node, other_node)
                merged_nodes.add(other_node)
                break  # Stop after merging with one node

    # Set zero radii to 0.1
    for node in G.nodes():
        if G.nodes[node]['radius'] == 0:
            G.nodes[node]['radius'] = 0.1

    return G

G = cleanup_graph(G)

In [4]:
#@title Define G = test graph
# Import the necessary library
G = FenicsGraph()

# Define the node labels and their corresponding integer IDs
node_mapping = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'E',
    5: 'F',
    6: 'G',
    7: 'H'
}

# Define the coordinates for each node (x, y, z)
node_coords = {
    0: [0, 20, 15],    # A
    1: [10, 20, 15],   # B
    2: [22, 13, 15],   # C
    3: [22, 28, 15],   # D
    4: [15, 5, 15],    # E
    5: [15, 35, 15],   # F
    6: [38, 5, 15],    # G
    7: [38, 35, 15]    # H
}

# Define the edges along with their radii
edges_with_radii = [
    (0, 1, 4),  # AB
    (1, 2, 3),  # BC
    (1, 3, 3),  # BD
    (2, 4, 2),  # CE
    (2, 6, 3),  # CG
    (3, 5, 2),  # DF
    (3, 7, 3)   # DH
]

# Create the FenicsGraph object
G = FenicsGraph()

# Add nodes to the graph with their positions
for node_id, coord in node_coords.items():
    G.add_node(node_id, pos=coord)

# Add edges to the graph with their radii
for u, v, radius in edges_with_radii:
    G.add_edge(u, v, radius=radius)

In [17]:
importlib.reload(FEMSink)

# k = 9.44e-3
# mu_o = 4.5e-3
P_cvp = 1 #mmHg
P_in = 100 #mmHg

test = FEMSink.FEMSink(
  G,
  kappa = 1.0e+4,
  alpha = 1.0e-10,
  beta = 1.0e-3,
  gamma = 1.0,
  P_inlet = P_in * 133.322, # Pa
  theta = 1.0,
  P_sink = P_cvp * 133.322, # Pa
  Lambda_endpoints = [0],
  Omega_box = [0, 0, 0, 40, 40, 30]
)

Averaging over 14 cells: 100%|██████████| 14/14 [00:00<00:00, 626.34it/s]


In [16]:
test.save_vtk(os.path.join(WD_PATH, 'perfusion_results', 'test_mda1'))